In [ ]:
!pip install Keras-Applications
!pip install tensorflow

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sn; sn.set(font_scale = 1.4)

import tensorflow as tf                
from sklearn.utils import shuffle 
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

import os
import cv2

In [2]:
import sys
import tensorflow as tf
from tensorflow.python.platform import build_info
import numpy as np


print("Python version:", sys.version)
print("NumPy version:", np.__version__)
print("TensorFlow version:", tf.__version__)
print("Keras version:", tf.keras.__version__)
print("CUDA version:", build_info.cuda_version)
print("cuDNN version:", build_info.cudnn_version)


Python version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
NumPy version: 1.26.4
TensorFlow version: 2.18.0
Keras version: 3.6.0


AttributeError: module 'tensorflow.python.platform.build_info' has no attribute 'cuda_version'

In [6]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("TensorFlow version:", tf.__version__)


Num GPUs Available:  0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10224368625669993791
xla_global_id: -1
]
TensorFlow version: 2.18.0


In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[0], 'GPU')

In [ ]:
EPOCHS = 50
BS = 64
DROPOUT_RATE = 0.25
FROZEN_LAYER_NUM = 170

ADAM_LEARNING_RATE = 0.001
SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001

Resize_pixelsize = 197

In [ ]:
classes = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
class_label = {classes: i for i, classes in enumerate(classes)}

no_of_classes = len(classes)

IMAGE_SIZE = (150, 150)

print("No. of classes: ", no_of_classes)

In [ ]:
base_dir = r'C:\Users\rafid\OneDrive\Desktop\VSCode\Python\Image Classification\Datasets\Intel Image Classification\seg_train\seg_train'

rows = 6
cols = 4
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(cols * 4, rows * 6)

valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

all_images = []
for class_name in classes:
    class_path = os.path.join(base_dir, class_name)
    image_paths = [os.path.join(class_path, fname) for fname in os.listdir(class_path)
                   if fname.lower().endswith(valid_extensions)][pic_index:pic_index + 4]
   
    all_images.extend(image_paths)
                       

for i, img_path in enumerate(all_images):
  sp = plt.subplot(rows, cols, i + 1)
  sp.axis('Off')
  img = mpimg.imread(img_path)
  plt.imshow(img)

  if i % 4 == 0:
      sp.title.set_text(classes[i // 4])

  
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    shear_range = 0.2,
    zoom_range = 0.2,
    validation_split = 0.1)

train_generator = train_datagen.flow_from_directory(
    'C:/Users/rafid/OneDrive/Desktop/VSCode/Python/Image Classification/Datasets/Intel Image Classification/seg_train/seg_train/',
    subset = 'training',
    target_size = (150, 150),
    batch_size = BS,
    class_mode = 'sparse',
    seed = 1,
    color_mode = 'rgb',
    shuffle=True)

validation_generator = train_datagen.flow_from_directory(
    'C:/Users/rafid/OneDrive/Desktop/VSCode/Python/Image Classification/Datasets/Intel Image Classification/seg_train/seg_train/',
    subset = 'validation',
    target_size = (150, 150),
    batch_size = BS,
    class_mode = 'sparse',
    seed = 1,
    color_mode = 'rgb',
    shuffle=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
    'C:/Users/rafid/OneDrive/Desktop/VSCode/Python/Image Classification/Datasets/Intel Image Classification/seg_test/seg_test/',
    target_size = (150, 150),
    batch_size = BS,
    class_mode = 'sparse',
    seed = 1,
    color_mode = 'rgb',
    shuffle = True)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(DROPOUT_RATE))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(DROPOUT_RATE))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(DROPOUT_RATE))

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(DROPOUT_RATE))

model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(DROPOUT_RATE))

model.add(Dense(6, activation='softmax'))

rlrop = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', factor=0.5, patience=20, min_lr= 0.00001, verbose=1)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', min_delta=0, patience=20, restore_best_weights=True, verbose=2)

callbacks_list = [rlrop, early_stop]

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    x = train_generator,
    validation_data = validation_generator,
    shuffle = True,
    epochs = EPOCHS,
    steps_per_epoch = 12632//BS,
    validation_steps =  1402//BS,
    callbacks = callbacks_list,
)